<a href="https://colab.research.google.com/github/vin-thomas/Enron_email_classification/blob/main/Tfidf_Enron__Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import torch
import glob
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/enron_csv_1', index_col=[0])

##**Extracting Emails**

In [ ]:
import re

add_fm = []
add_to = []
add_cc = []
add_bcc = []
for item in df.Raw_Text:
  fm_emails = re.findall(r"From: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  to_emails = re.findall(r"To: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  cc_emails = re.findall(r"Cc: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  bcc_emails = re.findall(r"Bcc: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  if fm_emails:
    add_fm.append(fm_emails[0])
  else:
    add_fm.append(1000)
  if to_emails:
    add_to.append(to_emails[0])
  else:
    add_to.append(1000)
  if cc_emails:
    add_cc.append(cc_emails[0])
  else:
    add_cc.append(1000)
  if bcc_emails:
    add_bcc.append(bcc_emails[0])
  else:
    add_bcc.append(1000)  

In [ ]:
df['From']= add_fm
df['To']= add_to
df['CC']= add_cc
df['BCC']= add_bcc

##**Extracting Dates**

In [ ]:
from datetime import datetime

def preprocess_date(date):
  format_data = "%a, %d %b %Y"
  date = datetime.strptime(date, format_data)
  return date


In [ ]:
date_email =[]

for item in df.Raw_Text:
  date = re.findall(r"Date: ([a-zA-Z]{3}, \d+ [a-zA-Z]{3} \d{4})", str(item))
  date= preprocess_date(date[0])
  date_email.append(date)


In [ ]:
df['date']= date_email

##**Extracting Subject**

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\\', '', text)
    text = re.sub(r'\\n', '', text)
    text = re.sub(r'-', '', text)
    return text

In [ ]:
subject_email = []

for item in df.Raw_Text:
  subject = re.findall(r"Subject: ([^\n]*|SUBJECT: [^\n]*)", str(item))
  subject = preprocess_text(subject[0])
  subject_email.append(subject)


In [ ]:
df['Subject']= subject_email

##**Extracting Body**

In [ ]:
body_email =[]
for item in df.Raw_Text:
  body = re.findall (r"FileName:([\s\S]*)$", str(item))
  body = preprocess_text(body[0])
  word_list = body.split(' ')
  word_list = word_list[2:130]
  if len(word_list)<128:
    word_list = ['na']
  word_list = " ".join(word_list)
  body_email.append(word_list)


In [ ]:
df['Body']= body_email

In [ ]:
df_final = df.drop(['Raw_Text'], axis=1)

In [ ]:
df_final

,Name,Folder,From,To,CC,BCC,date,Subject,Body
0,sager-e,notes_inbox,enron.announcements@enron.com,all.states@enron.com,1000,1000,2000-12-12 00:00:00,improved process for engaging temporary worker...,"you are aware, enron utilizes temporary staffi..."
1,sager-e,notes_inbox,veronica.gonzalez@enron.com,elizabeth.sager@enron.com,1000,1000,2001-06-07 00:00:00,margin letter socalrnmimeversion: .rncontentt...,na
2,sager-e,notes_inbox,arsystem@mailman.enron.com,elizabeth.sager@enron.com,1000,1000,2001-06-07 00:00:00,your approval is overdue: access request for s...,na
3,sager-e,notes_inbox,arsystem@mailman.enron.com,elizabeth.sager@enron.com,1000,1000,2001-06-07 00:00:00,your approval is overdue: access request forrn...,na
4,sager-e,notes_inbox,lech.kalembka@cwt.com,elizabeth.sager@enron.com,david.mitchell@cwt.com,david.mitchell@cwt.com,2001-05-04 00:00:00,setoff provisionsrncc: david.mitchell@cwt.comr...,have attached comments on the provisions you s...
...,...,...,...,...,...,...,...,...,...
517396,hain-m,all_documents,shapp@caiso.com,20participants@caiso.com,1000,1000,2001-03-01 00:00:00,caiso notice: request for cost information for...,na
517397,hain-m,all_documents,rrga-l@list.rtowest.org,rrga-l@list.rtowest.org,1000,1000,2001-05-18 00:00:00,rnmimeversion: .rncontenttype: text/plain; cha...,"shauna.nnto unsubscribe from the list, please ..."
517398,hain-m,all_documents,mary.hain@enron.com,lysa.akin@enron.com,1000,1000,2000-07-26 00:00:00,"re: washington, dc department meetingrnmimever...",thanksnlysa akin|pdx|ect wrote on // : am:nn>g...
517399,hain-m,all_documents,customersupport@factiva.rsc01.com,mary.hain@enron.com,1000,1000,2001-06-07 00:00:00,good news about the financial times and dow jo...,subscriber:nnnin our continuous commitment to ...


In [ ]:
df_final= df_final.drop_duplicates(subset=['Body', 'From'])

In [ ]:
df_final = df_final[df_final.Folder != 'all_documents']
df_final = df_final[df_final.Folder != 'deleted_items']
df_final = df_final[df_final.Body != 'na']

In [ ]:
df_final['Text']= df_final['Subject']+df_final['Body']
df_final.drop(['Subject', 'Body'], axis=1, inplace= True)

In [ ]:
df_final

,Name,Folder,From,To,CC,BCC,date,Text
0,sager-e,notes_inbox,enron.announcements@enron.com,all.states@enron.com,1000,1000,2000-12-12 00:00:00,improved process for engaging temporary worker...
4,sager-e,notes_inbox,lech.kalembka@cwt.com,elizabeth.sager@enron.com,david.mitchell@cwt.com,david.mitchell@cwt.com,2001-05-04 00:00:00,setoff provisionsrncc: david.mitchell@cwt.comr...
6,sager-e,notes_inbox,janet.moore@enron.com,elizabeth.sager@enron.com,1000,1000,2001-05-10 00:00:00,draft eei credit annexrnmimeversion: .rnconten...
8,sager-e,notes_inbox,christi.nicolay@enron.com,tom.chapman@enron.com,tom.hoatson@enron.com,tom.hoatson@enron.com,2001-04-04 00:00:00,pa wholesale vs. retail pricesrncc: tom.hoatso...
9,sager-e,notes_inbox,david.portz@enron.com,fmdutton@aep.com,tracy.ngo@enron.com,tracy.ngo@enron.com,2000-12-08 00:00:00,re: eei cover sheet completionrncc: tracy.ngo@...
...,...,...,...,...,...,...,...,...
515891,hain-m,discussion_threads,lpeters@pacifier.com,mary.hain@enron.com,1000,1000,2001-03-26 00:00:00,re: life after enronrnmimeversion: .rncontentt...
515951,hain-m,discussion_threads,tshuba@sheagardner.com,elchristensen@snopud.com,pdmichie@bpa.gov,pdmichie@bpa.gov,2000-06-25 00:00:00,re: initial email communication subgroup work...
515959,hain-m,discussion_threads,atrodrigues@bpa.gov,mary.hain@enron.com,1000,1000,2001-03-25 00:00:00,re: i'm leaving enronrnmimeversion: .rncontent...
516002,hain-m,discussion_threads,lysa.akin@enron.com,1000,1000,1000,2001-03-19 00:00:00,ron carrol call pin #: (full details below)rn...


In [ ]:
df_final['from_id']= df_final['From'].factorize()[0]
df_final['to_id']= df_final['To'].factorize()[0]
df_final['cc_id']= df_final['CC'].factorize()[0]
df_final['bcc_id']= df_final['BCC'].factorize()[0]
df_final['folder']= df_final['Folder'].factorize()[0]
df_final['name']= df_final['Name'].factorize()[0]

In [ ]:
df_final.drop(['Name', 'Folder', 'From', 'To', 'CC', 'BCC', 'date'], axis=1, inplace= True)

In [ ]:
df_tfidf= df_final.sample(60000)

In [ ]:
df_tfidf

,Text,from_id,to_id,cc_id,bcc_id,folder,name
449128,draft background and q&a for ca residential c...,716,39,630,625,9,192
317767,re: u.s. refundrnmimeversion: .rncontenttype: ...,5943,5727,0,0,6,141
208366,re: fw: directorlevel impact and influencernmi...,285,926,0,0,8,91
324898,"re: letter agreementrncc: kay.mann@enron.com, ...",32,7406,2091,2058,8,143
98436,june transport usage ticketsrncc: mary.smith@e...,2966,955,1458,1441,230,49
...,...,...,...,...,...,...,...
320237,re: west gas trading rotationrnmimeversion: .r...,295,1313,0,0,6,142
249785,fw: medicine bow tiein capacityrnmimeversion: ...,394,271,0,0,6,115
362266,re: fw: pg&e energy trading entitiesrnmimevers...,275,1852,0,0,0,160
90795,thoughts on upcoming visitrncc: tammie.schoppe...,2658,2692,980,972,88,43


In [ ]:
df_tfidf = df_tfidf[["Text", "from_id", "to_id", "cc_id", "bcc_id", "name", "folder"]]

In [ ]:
df_tfidf= df_tfidf.reset_index(drop=True)

In [ ]:
df_tfidf

,Text,from_id,to_id,cc_id,bcc_id,name,folder
0,draft background and q&a for ca residential c...,716,39,630,625,192,9
1,re: u.s. refundrnmimeversion: .rncontenttype: ...,5943,5727,0,0,141,6
2,re: fw: directorlevel impact and influencernmi...,285,926,0,0,91,8
3,"re: letter agreementrncc: kay.mann@enron.com, ...",32,7406,2091,2058,143,8
4,june transport usage ticketsrncc: mary.smith@e...,2966,955,1458,1441,49,230
...,...,...,...,...,...,...,...
59995,re: west gas trading rotationrnmimeversion: .r...,295,1313,0,0,142,6
59996,fw: medicine bow tiein capacityrnmimeversion: ...,394,271,0,0,115,6
59997,re: fw: pg&e energy trading entitiesrnmimevers...,275,1852,0,0,160,0
59998,thoughts on upcoming visitrncc: tammie.schoppe...,2658,2692,980,972,43,88


##**Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2000, max_df=60000, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df_tfidf['Text']).toarray()


In [ ]:
type(features), features.shape

(numpy.ndarray, (60000, 1060))

In [ ]:
df_features = pd.DataFrame(features)

In [ ]:
df_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.129703,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.080196,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.086521,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.087825,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.099393,0.0,0.0,0.000000,0.0,0.0,0.0,0.155159,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.092216,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.142164,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.065374,0.091083,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.120976,0.000000,0.0,0.0
3,0.000000,0.000000,0.125771,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.117005,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.073504,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.185816,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.200721,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.283537,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.093118,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.303620,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
59996,0.000000,0.068436,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.141377,0.0,0.000000,0.0,0.116244,0.094216,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000

In [ ]:
df_tfidf.drop(['Text'], axis=1, inplace= True)

In [ ]:
df_tfidf= df_tfidf.join(df_features)

In [ ]:
df_tfidf

,from_id,to_id,cc_id,bcc_id,name,folder,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,...,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059
0,716,39,630,625,192,9,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.129703,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.086521,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,5943,5727,0,0,141,6,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.087825,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.099393,0.0,0.0,0.000000,0.0,0.0,0.0,0.155159,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.142164,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,285,926,0,0,91,8,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.065374,0.091083,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.120976,0.000000,0.0,0.0
3,32,7406,2091,2058,143,8,0.000000,0.000000,0.125771,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.117005,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,2966,955,1458,1441,49,230,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.185816,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.200721,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.283537,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,295,1313,0,0,142,6,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.303620,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
59996,394,271,0,0,115,6,0.000000,0.068436,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.141377,0.0,0.000000,0.0,0.116244,0.094216,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.20826,0.0,0.0,0.0,0.000000,0.0,0.00

In [ ]:
y = df_tfidf.folder
X= df_tfidf.drop(['folder'], axis=1)

In [ ]:
X.shape

(60000, 1065)

##**Standardization**

In [ ]:
from sklearn.preprocessing import StandardScaler


# perform a robust scaler transform of the dataset
ss = StandardScaler()
X = ss.fit_transform(X)
# convert the array back to a dataframe
X = pd.DataFrame(X)
X.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064
0,-0.704921,-0.958429,0.205134,0.223660,1.322993,-0.174483,-0.324036,-0.210051,-0.188841,-0.280136,-0.175641,-0.214967,-0.163972,-0.220083,-0.171817,-0.186274,4.689202,-0.199552,-0.24582,-0.300212,-0.318222,-0.175655,-0.239717,-0.203132,-0.194464,-0.178177,-0.174503,-0.211404,-0.203817,-0.205309,-0.214839,-0.339835,-0.181795,-0.164731,-0.208363,-0.204414,-0.214905,-0.284812,-0.195752,-0.195287,...,-0.223828,-0.233646,-0.195804,-0.199275,-0.23483,-0.222184,-0.183621,-0.448598,-0.252501,-0.188136,3.613528,-0.361501,-0.234107,-0.188455,-0.323852,-0.449743,-0.21686,-0.242403,-0.184498,-0.226924,-0.204663,-0.198014,-0.224307,-0.274613,-0.199017,-0.195027,-0.170037,-0.44501,-0.169634,-0.335576,-0.189617,-0.236025,-0.1646,-0.347806,-0.186974,-0.176443,-0.385942,-0.287544,-0.231909,-0.224052
1,1.112361,1.046073,-0.424543,-0.420967,0.517451,-0.174483,-0.324036,-0.210051,-0.188841,-0.280136,-0.175641,-0.214967,-0.163972,3.221569,-0.171817,-0.186274,-0.218790,-0.199552,-0.24582,-0.300212,-0.318222,4.527797,-0.239717,-0.203132,-0.194464,-0.178177,-0.174503,-0.211404,6.194926,-0.205309,-0.214839,-0.339835,-0.181795,-0.164731,-0.208363,-0.204414,-0.214905,-0.284812,-0.195752,-0.195287,...,-0.223828,-0.233646,-0.195804,-0.199275,-0.23483,-0.222184,-0.183621,-0.448598,-0.252501,-0.188136,-0.167110,-0.361501,-0.234107,-0.188455,-0.323852,3.354391,-0.21686,-0.242403,-0.184498,-0.226924,-0.204663,-0.198014,-0.224307,-0.274613,-0.199017,-0.195027,-0.170037,-0.44501,-0.169634,-0.335576,-0.189617,-0.236025,-0.1646,-0.347806,-0.186974,-0.176443,-0.385942,-0.287544,-0.231909,-0.224052
2,-0.854768,-0.645842,-0.424543,-0.420967,-0.272297,-0.174483,-0.324036,-0.210051,-0.188841,-0.280136,-0.175641,-0.214967,-0.163972,-0.220083,-0.171817,-0.186274,-0.218790,-0.199552,-0.24582,-0.300212,-0.318222,-0.175655,-0.239717,-0.203132,-0.194464,-0.178177,-0.174503,-0.211404,-0.203817,-0.205309,-0.214839,-0.339835,-0.181795,-0.164731,-0.208363,-0.204414,-0.214905,-0.284812,-0.195752,-0.195287,...,-0.223828,-0.233646,-0.195804,-0.199275,-0.23483,-0.222184,-0.183621,1.394094,2.711356,-0.188136,-0.167110,-0.361501,-0.234107,-0.188455,-0.323852,-0.449743,-0.21686,-0.242403,-0.184498,-0.226924,-0.204663,-0.198014,-0.224307,-0.274613,-0.199017,-0.195027,-0.170037,-0.44501,-0.169634,-0.335576,-0.189617,-0.236025,-0.1646,-0.347806,-0.186974,-0.176443,2.934232,-0.287544,-0.231909,-0.224052
3,-0.942729,1.637767,1.665386,1.701661,0.549040,-0.174483,-0.324036,7.804925,-0.188841,-0.280136,-0.175641,-0.214967,-0.163972,-0.220083,-0.171817,-0.186274,-0.218790,-0.199552,-0.24582,-0.300212,-0.318222,-0.175655,-0.239717,-0.203132,-0.194464,-0.178177,-0.174503,-0.211404,-0.203817,-0.205309,-0.214839,2.459048,-0.181795,-0.164731,-0.208363,-0.204414,-0.214905,-0.284812,-0.195752,-0.195287,...,-0.223828,-0.233646,-0.195804,-0.199275,-0.23483,-0.222184,-0.183621,-0.448598,-0.252501,-0.188136,-0.167110,-0.361501,-0.234107,-0.188455,-0.323852,-0.449743,-0.21686,-0.242403,-0.184498,-0.226924,-0.204663,-0.198014,-0.224307,-0.274613,-0.199017,-0.195027,-0.170037,-0.44501,-0.169634,-0.335576,-0.189617,-0.236025,-0.1646,-0.347806,-0.186974,-0.176443,-0.385942,-0.287544,-0.231909,-0.224052
4,0.077341,-0.635622,1.032710,1.065285,-0.935685,-0.174483,-0.324036,-0.210051,-0.188841,-0.280136,-0.175641,-0.214967,-0.163972,-0.220083,-0.171817,-0.186274,6.812539,-0.199552,-0.24582,-0.300212,-0.318222,-0.175655,-0.239717,-0.203132,8.008083,-0.178177,-0.174503,-0.211404,-0.203817,-0.205309,-0.214839,-0.339835,-0.181795,-0.164731,-0.208363,-0.204414,-0.214905,-0.284812,-0.195752,-0.195287,...,-0.223828,-0.233646,-0.195804,-0.199275,-0.23483,-0.222184,9.369482,-0.448598,-0.252501,-0.188136,-0.16711

##**Train/Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_test.shape, y_test.shape

((12000, 1065), (12000,))

##**Train**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
models = [
          ('KNN', KNeighborsClassifier()),  
          ('Random Forest', RandomForestClassifier(n_estimators=100, max_depth=3 )), ('Gradient Descent',SGDClassifier())
         ]
for name, model in models:
  model.fit(X_train, y_train)
  y_pred= model.predict(X_test)
  accuracy= accuracy_score(y_test, y_pred)
  print (name, accuracy)

KNN 0.6404166666666666
Random Forest 0.7268333333333333
Gradient Descent 0.81025


In [ ]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.8, random_state=0)

model = SVC()
model.fit(X_train_, y_train_)
y_pred_= model.predict(X_test_)
accuracy= accuracy_score(y_test_, y_pred_)
print ('SVC', accuracy)

SVC 0.8031666666666667


In [ ]:
model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)
y_pred= model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print ('LR', accuracy)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LR 0.883
